<a href="https://colab.research.google.com/github/ambideXtrous9/PyTorch-Models/blob/main/BIAS_LSTM_PYTORCH_LIGHTNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import warnings
import torch
from torch.utils.data import DataLoader
from torch import nn
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning import Trainer
warnings.filterwarnings(action='ignore',category=UserWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)
from torch.utils.data import TensorDataset, DataLoader
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sushovan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
glove_path = 'glove.6B.100d.txt'

In [ ]:
def Sentence2Vec(filename,glovepath,embedding_dim = 100,max_length = 120):
    df_train = pd.read_excel(filename)
    T1 = df_train['content_original'].str.split(' \n\n---\n\n').str[0]
    df_train['content_original'] = T1.str.replace('-',' ').str.replace('[^\w\s]','').str.replace('\n',' ').str.lower()
    df = df_train[['content_original','source','bias_text','bias']]
    T = df['content_original']
    stop = stopwords.words('english')
    T = T.apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
    path = glovepath
    tokenizer = Tokenizer()
    text=T
    tokenizer.fit_on_texts(text)
    word_index=tokenizer.word_index
    print("number of word in vocabulary",len(word_index))
    vocab_size = 5000
    trunc_type = 'post'
    oov_tok = '<OOV>'
    padding_type = 'post'
    #print("words in vocab",word_index)
    text_sequence=tokenizer.texts_to_sequences(text)
    text_sequence = pad_sequences(text_sequence, maxlen=max_length, truncating=trunc_type)
    print("word in sentences are replaced with word ID",text_sequence)
    size_of_vocabulary=len(tokenizer.word_index) + 1
    print("The size of vocabulary ",size_of_vocabulary)
    embeddings_index = dict()

    f = open(path)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))

    embedding_matrix = np.zeros((size_of_vocabulary, embedding_dim))

    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    text_shape = text_sequence.shape
    X_train = np.empty((text_shape[0],text_shape[1],embedding_matrix.shape[1]))
    for i in range(text_sequence.shape[0]):
        for j in range(text_sequence.shape[1]):
            X_train[i,j,:] = embedding_matrix[text_sequence[i][j]]
    print(X_train.shape)

    y_train = df['bias'].to_numpy()

    return X_train,y_train


In [ ]:
X_train,y_train = Sentence2Vec(filename='Train.xlsx',glovepath=glove_path, embedding_dim=100, max_length=120)

number of word in vocabulary 143699                                
word in sentences are replaced with word ID [[   3    2   24 ...    9  114 1996]
 [9796  590 2684 ...   61 1065 3100]
 [1958 2002 7091 ... 7257 2133 1951]
 ...
 [   2  900   94 ...  156  882   19]
 [   2   58 1794 ...   57   47   84]
 [3262 3233 4319 ...  472  382 2286]]
The size of vocabulary  143700
Loaded 400000 word vectors.
(26590, 120, 100)


In [ ]:
X_val,y_val = Sentence2Vec(filename='Test.xlsx',glovepath=glove_path,embedding_dim=100, max_length=120)

number of word in vocabulary 37490
word in sentences are replaced with word ID [[   15  2444  1170 ...  6866  6867   292]
 [ 1849  1254 17674 ... 10294   477   271]
 [   66  3773   922 ...   352  1242    17]
 ...
 [  252   610  3654 ...  2282  1443   403]
 [   36    49    73 ...  4708   219   997]
 [  452   312   956 ...    12   394     6]]
The size of vocabulary  37491
Loaded 400000 word vectors.
(1300, 120, 100)


In [ ]:
BS = 16

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [ ]:
train_features = torch.Tensor(X_train)
train_targets = torch.Tensor(y_train).type(torch.LongTensor)
val_features = torch.Tensor(X_val)
val_targets = torch.Tensor(y_val).type(torch.LongTensor)


trainDataset = TensorDataset(train_features, train_targets)
valDataset = TensorDataset(val_features, val_targets)

In [ ]:
class LitLSTM(pl.LightningModule):
    def __init__(self,num_classes,dimension,hidd_dim):
        super(LitLSTM, self).__init__()
        
        self.hidd_dim = hidd_dim
        
        self.LSTM = nn.LSTM(input_size=dimension,
                            hidden_size=self.hidd_dim,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=False)
        
        self.model = nn.Sequential( nn.Dropout(p=0.5),
                                    nn.ReLU(inplace=True),
                                    nn.BatchNorm1d(self.hidd_dim),
                                    nn.Linear(self.hidd_dim, num_classes),
                                    nn.LogSoftmax(dim=1))
        
        # add metrics
        self.train_acc = torchmetrics.Accuracy()
        self.train_f1 = torchmetrics.F1Score(number_classes=num_classes,average="micro")
        self.val_acc = torchmetrics.Accuracy()
        self.val_f1 = torchmetrics.F1Score(number_classes=num_classes,average="micro")
        
    def forward(self, x):
        h_0 = torch.tensor(torch.rand(2, BS, self.hidd_dim)).to(device)
        c_0 = torch.tensor(torch.rand(2, BS, self.hidd_dim)).to(device)
        out, (final_hidden_state, final_cell_state) = self.LSTM(x,(h_0,c_0))
        out = self.model(final_hidden_state[-1])
        return out

    def training_step(self, batch, batch_idx):
        images, labels = batch
        # Forward pass
        outputs = self.forward(images)
        lossfn = nn.NLLLoss()
        loss = lossfn(outputs, labels)
        
        y_pred = torch.exp(outputs)
        #y_pred = output.data.max(1, keepdim=True)[1]
        acc = self.train_acc(y_pred, labels)
        f1 = self.train_f1(y_pred, labels)
        # just accumulate

        self.log("train_loss", loss)
        self.log("train_accuracy", acc)
        self.log("train_f1", f1)
        tensorboard_logs = {'train_loss': loss}
        # use key 'log'
        return {"loss": loss, 'log': tensorboard_logs}

    # define what happens for testing here

    def train_dataloader(self):
        trainDataLoader = DataLoader(trainDataset, num_workers=2,batch_size=BS, shuffle=True,drop_last=True)

        return trainDataLoader

    def val_dataloader(self):
        valDataLoader = DataLoader(valDataset, num_workers=2,batch_size=BS,shuffle=False,drop_last=True)
        
        return valDataLoader
    
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        # Forward pass
        outputs = self.forward(images)
        lossfn = nn.NLLLoss()
        loss = lossfn(outputs, labels)
        
        pred = torch.exp(outputs)
        #pred = output.data.max(1, keepdim=True)[1]
        self.val_acc.update(pred, labels)
        self.val_f1.update(pred, labels)

        self.log("val_loss", loss)
        return {"val_loss": loss}
    
    def training_epoch_end(self, training_step_outputs):
        # compute metrics
        train_accuracy = self.train_acc.compute()
        train_f1 = self.train_f1.compute()
        # log metrics
        self.log("epoch_train_accuracy", train_accuracy)
        self.log("epoch_train_f1", train_f1)
        # reset all metrics
        self.train_acc.reset()
        self.train_f1.reset()
        print(f"\ntraining accuracy: {train_accuracy:.4}, "\
        f"f1: {train_f1:.4}")
        
    def validation_epoch_end(self, outputs):
        # outputs = list of dictionaries
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        val_accuracy = self.val_acc.compute()
        val_f1 = self.val_f1.compute()
        # log metrics
        self.log("val_accuracy", val_accuracy)
        self.log("val_loss", avg_loss)
        self.log("val_f1", val_f1)
        # reset all metrics
        self.val_acc.reset()
        self.val_f1.reset()
        print(f"\nvalidation accuracy: {val_accuracy:.4} "\
        f"f1: {val_f1:.4}")
        
        tensorboard_logs = {'avg_val_loss': avg_loss}
        # use key 'log'
        return {'val_loss': avg_loss, 'log': tensorboard_logs}
    
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

In [ ]:
# seeding for torch, numpy, stdlib random, including DataLoader workers!
seed_everything(123, workers=True)

early_stopping = EarlyStopping(
    monitor="val_loss",
    stopping_threshold=1e-6,
    divergence_threshold=9.0,
    check_finite=True)

Global seed set to 123


In [ ]:
model = LitLSTM(dimension=X_train.shape[2],hidd_dim=100,num_classes=3)
trainer = Trainer(accelerator='gpu',devices=1,max_epochs=100,log_every_n_steps=8,callbacks=[early_stopping])
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | LSTM      | LSTM       | 161 K 
1 | model     | Sequential | 503   
2 | train_acc | Accuracy   | 0     
3 | train_f1  | F1Score    | 0     
4 | val_acc   | Accuracy   | 0     
5 | val_f1    | F1Score    | 0     
-----------------------------------------
162 K     Trainable params
0         Non-trainable params
162 K     Total params
0.648     Total estimated model params size (MB)


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 60.19it/s]
validation accuracy: 0.5625 f1: 0.5625
Epoch 0: 100%|██████████| 1742/1742 [00:44<00:00, 38.89it/s, loss=1.09, v_num=3]
validation accuracy: 0.348 f1: 0.348
Epoch 0: 100%|██████████| 1742/1742 [00:44<00:00, 38.89it/s, loss=1.09, v_num=3]
training accuracy: 0.3778, f1: 0.3778
Epoch 1: 100%|██████████| 1742/1742 [00:45<00:00, 38.53it/s, loss=1.09, v_num=3]
validation accuracy: 0.3997 f1: 0.3997
Epoch 1: 100%|██████████| 1742/1742 [00:45<00:00, 38.52it/s, loss=1.09, v_num=3]
training accuracy: 0.3885, f1: 0.3885
Epoch 2: 100%|██████████| 1742/1742 [00:44<00:00, 39.07it/s, loss=0.989, v_num=3]
validation accuracy: 0.4552 f1: 0.4552
Epoch 2: 100%|██████████| 1742/1742 [00:44<00:00, 39.06it/s, loss=0.989, v_num=3]
training accuracy: 0.4144, f1: 0.4144
Epoch 3: 100%|██████████| 1742/1742 [00:46<00:00, 37.65it/s, loss=0.861, v_num=3]
validation accuracy: 0.4792 f1: 0.4792
Epoch 3: 100%|██████████| 1742/1742 [00:46<00:0

In [ ]:
%tensorboard --logdir lightning_logs

UsageError: Line magic function `%tensorboard` not found.
